In [247]:
from read_write_files import read_json, save_json
from collections import Counter
import itertools
import pandas as pd

In [97]:
def create_sense_zip(alignments):
    attr = "Sense"
    attr_zip = []
    for alignment in alignments:
        line_attr = []
        line_attr += [alignment["gold"][attr][0]]
        
        for pars in alignment["parsers"]:
            if pars == None:
                line_attr += ["None"]
            else:
                line_attr += [pars[attr][0]]
        
        attr_zip += [tuple(line_attr)]
    return attr_zip,alignment["parser_names"]

In [128]:
def sense_statistics(sense_comparison,parser_names,not_mapped_rels):
    zip_sense_comparison = list(zip(*sense_comparison))
    gold_senses = zip_sense_comparison[0]
    #Count all combination of senses
    sense_counter = Counter(sense_comparison)
    #Different Senses
    set_senses = set(gold_senses)
    #How many parsers will be compared
    len_parser = len(zip_sense_comparison)-1
    sense_rows = []
    
    #not mappable predicted relations
    not_mapped_counter = Counter([(rel["Parser"],rel["Sense"][0]) for rel in not_mapped_rels])

    for sense in set_senses:
        tmp_senses = set_senses.copy()
        tmp_senses.remove(sense)
        possible_comb = [[sense]] + [tmp_senses for i in range(len_parser)]
        diff_preds = itertools.product(*possible_comb)

        equal_correct_parsing = sense_counter[tuple([sense]+[sense]*len_parser)]

        all_wrong_parsing = sum([
            sense_counter[tuple(diff_pred)] 
            for diff_pred in diff_preds])

        total_act_sense_count = equal_correct_parsing + sum([1 for i in gold_senses if i == sense])


        equal_wrong_parsing = sum([
            sense_counter[tuple([sense]+[other_sense]*len_parser)] 
            for other_sense in tmp_senses])
        
        parser_columns = []

        #Parser is better than all the other
        parser_better = []
        for index_parser in range(1,len_parser+1):
            tmp_possible_comb = possible_comb[:]
            column_name = parser_names[index_parser-1]
            
            tmp_possible_comb[index_parser] = [sense]
            other_diff_preds = itertools.product(*tmp_possible_comb)
            
            pars_better = sum([sense_counter[other_diff_pred] for other_diff_pred in other_diff_preds])
            total_pred_sense_count = Counter(zip_sense_comparison[index_parser])[sense]#sum([1 for i in zip_sense_comparison[index_parser] if i == sense])
            
            not_mapped = not_mapped_counter[(column_name,sense)]
            
            right_pred_comb = [[sense]] + [set_senses]*len_parser
            right_pred_comb[index_parser] = [sense]
            right_pred = sum([sense_counter[comb] for comb in itertools.product(*right_pred_comb)])
            
            parser_better += [
                pars_better,
                not_mapped,
                right_pred,
                total_pred_sense_count+not_mapped]
            
            
            parser_columns += [string.format(column_name) for string in [
                "{}: unique right",
                "{}: not mapped",
                "{}: right pred",
                "{}: total pred"]]
            
        #Two Parser share opinions
        for parser1,parser2 in itertools.combinations([i for i in range(1,len_parser+1)],2):
            tmp_possible_comb = possible_comb[:]
            tmp_possible_comb[parser1] = [sense]
            tmp_possible_comb[parser2] = [sense]
            
            
            pars_better = sum([sense_counter[comb] for comb in itertools.product(*tmp_possible_comb)])
            total_pred_sense_count = sum(
                [1 for pars1_pred,pars2_pred in zip(*[
                    zip_sense_comparison[parser1],
                    zip_sense_comparison[parser2]
                ])
                if (pars2_pred == sense) != (pars1_pred == sense)])
            
            parser_better += [pars_better,total_pred_sense_count]
            
            column_name = " and ".join([parser_names[parser1-1],parser_names[parser2-1]])
            parser_columns += [
                "{} right".format(column_name),
                "Total Pred ({})".format(column_name)]
        

        #At least one parser is correct
        correct_sense_poss = [[sense]]+[set_senses]*len_parser
        combination_correct = sum(
            [sense_counter[at_least_one_right] 
             for at_least_one_right in itertools.product(*correct_sense_poss) if sense in at_least_one_right[1:]])

        sense_rows += [[sense,equal_correct_parsing,all_wrong_parsing,total_act_sense_count,equal_wrong_parsing,combination_correct]+parser_better]

    columns = ["Sense",
             "Equal Correct",
             "All Wrong",
             "Total Act",
             "Equal Wrong",
             "At least one correct"]+parser_columns
    return pd.DataFrame(
        columns=columns,
        data=sense_rows),columns


In [129]:
alignment_path = "data/project_files/total_alignment.json"
not_mappable_path = "data/project_files/not_mappable.json"

total_alignments = read_json(alignment_path)
not_mappable = read_json(not_mappable_path)

In [130]:
sense_zip,sense_zip_names = create_sense_zip(total_alignments)
sense_counter = Counter(sense_zip)

In [131]:

sense_df,df_columns = sense_statistics(sense_zip,sense_zip_names,not_mappable)

In [135]:
print("Overall Statistics")
sense_df[df_columns[:6]]

,Sense,Equal Correct,All Wrong,Total Act,Equal Wrong,At least one correct
0,Expansion.Conjunction,125,22,445,11,188
1,Temporal.Asynchronous.Precedence,14,10,64,5,19
2,Temporal.Synchrony,10,8,62,3,26
3,Temporal.Asynchronous.Succession,19,11,77,5,28
4,Contingency.Cause.Result,6,20,58,4,14
5,EntRel,29,12,229,1,112
6,Comparison.Concession,2,49,109,28,7
7,Contingency.Condition,12,0,38,0,13
8,Comparison.Contrast,10,14,64,6,19
9,Expansion.Instantiation,1,22,45,3,4


In [141]:
print("Parser Individual Statistics")
sense_df[df_columns[6:6+len(sense_zip_names)*4]]

Parser Individual Statistics


,oslopots: unique right,oslopots: not mapped,oslopots: right pred,oslopots: total pred,nguyenlab: unique right,nguyenlab: not mapped,nguyenlab: right pred,nguyenlab: total pred,steven: unique right,steven: not mapped,steven: right pred,steven: total pred
0,8,1,156,288,3,1,154,339,20,2,160,459
1,0,1,19,29,0,2,19,33,0,4,14,43
2,3,2,21,54,2,0,21,59,2,1,13,40
3,0,2,27,39,1,1,27,44,0,1,20,51
4,0,0,6,18,6,1,12,57,2,1,8,34
5,21,0,91,298,10,1,80,248,3,3,48,213
6,0,0,3,4,2,0,5,14,2,1,4,16
7,0,3,13,22,0,3,13,25,0,2,12,38
8,0,0,15,80,2,1,18,112,1,3,12,130
9,0,0,1,6,2,1,3,10,1,0,2,5


In [142]:
print("Two Parser Aggrement")
sense_df[df_columns[-len(sense_zip_names)*2:]]

Two Parser Aggrement


,oslopots and nguyenlab right,Total Pred (oslopots and nguyenlab),oslopots and steven right,Total Pred (oslopots and steven),nguyenlab and steven right,Total Pred (nguyenlab and steven)
0,17,237,6,380,9,347
1,5,13,0,37,0,36
2,8,31,0,47,1,50
3,7,20,1,43,0,35
4,0,56,0,35,0,65
5,33,243,8,298,8,255
6,1,12,0,15,0,23
7,1,9,0,25,0,24
8,5,57,0,113,1,128
9,0,13,0,7,0,12


# Model Training

In [228]:
def get_sense_lists(relations):
    gold_senses = []
    parser_senses = []
    parser_names = relations[0]["parser_names"]
    
    for rel in relations:
        gold_senses += [rel["gold"]["Sense"][0]]
        
        parser_pred = []
        for parser in rel["parsers"]:
            if parser == None:
                parser_pred += ["None"]
            else:
                parser_pred += [parser["Sense"][0]]
        
        parser_senses += tuple([parser_pred])
    parser_senses_zip = list(zip(*parser_senses))
    
    return gold_senses,parser_senses_zip,parser_names
        
            
    

In [206]:
def calc_prob_measure(gold_total,pred_total,tp):
    prec = 0
    if pred_total != 0:
        prec = tp/pred_total
        
    rec = 0
    if gold_total != 0:
        rec = tp/gold_total
    
    f1 = 0
    if prec != 0 and rec != 0:
        f1 = ((prec*rec)/(prec+rec))
    
    return prec,rec,f1

In [229]:
def create_one_parser_model(gold_senses,pred_senses):
    sense_model = dict()
    
    sense_set = set(gold_senses)
    sense_compare_zip = zip(*[gold_senses,pred_senses])
    sense_counter = Counter(sense_compare_zip)
    pred_counter = Counter(pred_senses)
    gold_counter = Counter(gold_senses)
    
    for sense in sense_set:
        gold_total = gold_counter[sense]
        pred_total = pred_counter[sense]
        
        prec,rec,f1 = calc_prob_measure(gold_total,pred_total,sense_counter[(sense,sense)])
        sense_model[sense] = {
            "prec":prec,
            "rec":rec,
            "f1":f1
        }
    return sense_model
    
    

In [231]:
gold_senses,parser_senses,parser_names = get_sense_lists(total_alignments)

In [248]:
model = []
model_path = "data/project_files/sense_model.json"
for parser_name,parser_pred in zip(*[parser_names,parser_senses]):
    tmp_model = create_one_parser_model(gold_senses,parser_pred)
    model += [{
        "parser":parser_name,
        "sense_pred": tmp_model
    }]
    
save_json(model,model_path)